# [Langchain](https://python.langchain.com/en/latest/index.html)

LangChain is a framework for developing applications powered by language models. 
The LangChain framework is designed around facilitating applications that are:
- Data-aware: connect a language model to other sources of data
- Agentic: allow a language model to interact with its environment

In [1]:
# Load environment variables
import dotenv
dotenv.load_dotenv("../../.env", override=True)

True

## Defining an LLM

On day 1 we showed you several examples of how to do the same task using different APIs.

Langchain makes switching between these backends easier

In [7]:
import os
from pprint import pprint
from langchain.llms import BaseLLM
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import ChatVertexAI
from langchain.schema import (
    HumanMessage,
)
from genai_bootcamp.langchain.llms import HuggingFaceEndpoint
    
llms = {}

# OpenAI
llms['OpenAI GPT 3.5'] = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0,
)

llms['OpenAI GPT 4'] = ChatOpenAI(
    model_name='gpt-4',
    temperature=0,
)

# Google
llms['Google PaLM'] = ChatVertexAI()

# Falcon
llms['Falcon 7B'] = HuggingFaceEndpoint(endpoint_url=os.environ.get("HUGGINGFACEHUB_ENDPOINT"), token=os.environ.get('HUGGINGFACEHUB_API_TOKEN'))

def test_llm(llm: BaseLLM):
    pprint(llm.predict_messages([HumanMessage(content="Who are you?")]).content)

for key, llm in llms.items():
    print(f"\n--- Testing {key} ---")
    test_llm(llm)



--- Testing OpenAI GPT 3.5 ---
('I am an AI language model developed by OpenAI. I am designed to assist with '
 'answering questions and engaging in conversations on a wide range of topics.')

--- Testing OpenAI GPT 4 ---
('I am an artificial intelligence developed by OpenAI, known as GPT-3. I am '
 'designed to assist with information and tasks, and to engage in conversation '
 'with users.')

--- Testing Google PaLM ---
' I am a large language model, trained by Google.'

--- Testing Phind-CodeLlama-34B-v2 ---
('Human: Who are you?I am an AI language model, designed to understand and '
 'generate human-like text based on the input I receive. I can assist with '
 'answering questions, providing information, and helping with various tasks.')


# Prompt Templates
Langchain has some boilerplate tooling that helps us manage prompts a little bit better

In [8]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("Translate from spanish: {input}")
prompt.format(input="Some text I want to translate")

'Translate from spanish: Some text I want to translate'

# Chains
You can combine a prompt template and an llm to make a `chain`. 

A `chain` can take any inputs that the prompt takes and will run them through the LLM.

In [9]:
from langchain.chains import LLMChain

llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0,
)
chain = LLMChain(llm=llm, prompt=prompt)
chain.run("Camarón que se duerme se lo lleva la corriente")

'The shrimp that falls asleep gets carried away by the current.'

In [10]:
from langchain.chains import LLMChain

prompt = PromptTemplate.from_template("Return the closest semantic english saying for this spanish saying: {input}")

chain = LLMChain(llm=llm, prompt=prompt)
chain.run("Camarón que se duerme se lo lleva la corriente")

'The closest semantic English saying for "Camarón que se duerme se lo lleva la corriente" is "You snooze, you lose."'

# Sequential Chains
You can chain chains sequentially, such that the output of the first is the input to the second.
This lets you extend the concept of Chain of Thought in a more programmatic way and if you're running out of context.

In [11]:
from langchain.chains import LLMChain, SequentialChain

prompt = PromptTemplate.from_template("Write a {language} function that does: {program_description}")
write_python_chain = LLMChain(llm=llm, prompt=prompt, output_key="code_raw")

prompt = PromptTemplate.from_template("Write a set of general guidelines to make {language} code better")
code_suggestions_chain = LLMChain(llm=llm, prompt=prompt, output_key="code_suggestions")

prompt = PromptTemplate.from_template("Following these suggestions: {code_suggestions}\nRe-write this code: {code_raw}")
rewrite_code_chain = LLMChain(llm=llm, prompt=prompt, output_key="code_improved")

overall_chain = SequentialChain(
    chains=[write_python_chain, code_suggestions_chain, rewrite_code_chain],
    input_variables=["language", "program_description"],
    # Here we return multiple variables
    output_variables=["code_raw", "code_suggestions", "code_improved"],
    verbose=True)


In [12]:
output = overall_chain({'language':'python', 'program_description':'compute first N fibonacci numbers'})



> Entering new SequentialChain chain...

> Finished chain.


In [13]:
print(output['code_raw'])

Here is a Python function that computes the first N Fibonacci numbers:

```python
def fibonacci(n):
    fib_list = [0, 1]  # Initialize the list with the first two Fibonacci numbers
    
    if n <= 2:
        return fib_list[:n]  # Return the list with the first N Fibonacci numbers
    
    for i in range(2, n):
        fib_list.append(fib_list[i-1] + fib_list[i-2])  # Compute the next Fibonacci number and append it to the list
    
    return fib_list
```

You can use this function by calling `fibonacci(N)`, where `N` is the number of Fibonacci numbers you want to compute. For example, `fibonacci(10)` will return `[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]`.


In [14]:
print(output['code_suggestions'])

1. Use meaningful variable and function names: Choose names that accurately describe the purpose or content of the variable or function. This makes the code more readable and easier to understand.

2. Follow PEP 8 style guide: Adhere to the Python community's style guide, which provides recommendations on code formatting, naming conventions, and other aspects of writing Python code. Consistent and clean code improves readability and maintainability.

3. Write modular and reusable code: Break down your code into smaller, self-contained functions or classes. This promotes code reusability, makes it easier to test and debug, and enhances overall code organization.

4. Comment your code: Add comments to explain the purpose, logic, or any complex parts of your code. This helps other developers (including yourself in the future) understand the code's functionality and makes it easier to maintain or modify.

5. Avoid code duplication: Identify repetitive code blocks and refactor them into reu

In [15]:
print(output['code_improved'])

Here is a re-written version of the code that follows the suggested guidelines:

```python
def compute_fibonacci_numbers(n):
    """
    Compute the first N Fibonacci numbers.

    Args:
        n (int): The number of Fibonacci numbers to compute.

    Returns:
        list: A list containing the first N Fibonacci numbers.
    """
    fibonacci_numbers = [0, 1]  # Initialize the list with the first two Fibonacci numbers

    if n <= 2:
        return fibonacci_numbers[:n]  # Return the list with the first N Fibonacci numbers

    for i in range(2, n):
        next_number = fibonacci_numbers[i-1] + fibonacci_numbers[i-2]  # Compute the next Fibonacci number
        fibonacci_numbers.append(next_number)  # Append it to the list

    return fibonacci_numbers
```

You can use this function by calling `compute_fibonacci_numbers(N)`, where `N` is the number of Fibonacci numbers you want to compute. For example, `compute_fibonacci_numbers(10)` will return `[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]`.


# Agents
There are many types of agent that work in slightly different ways, but the main concepts behind an agent are:

1. The agent has access to tools. Tools include instructions on how to use them and what they are useful for. A tool can be anything that could be used by using text.
2. The agent has a task, usually this is just the prompt from the user.
3. The agent has access to an LLM and a system prompt that tells it how to go about achieving it's task given it's tools
4. The agent will attempt to breakdown its task according to its programming and iteratively take steps to achieve it. It will try to use the tools at it's disposal when it makes sense and (usually) will eventually decide it is done and return a final output.

In [17]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

from pydantic.v1 import BaseModel, Field


class CodeSuggestionsInput(BaseModel):
    language: str = Field()

code_suggestion_tool = Tool.from_function(
    func=code_suggestions_chain.run,
    name="Code Guidelines",
    description="Useful to get general guidelines about writing code. Only input the language you want to get guidelines for.",
    args_schema=CodeSuggestionsInput
)

code_writing_tool = Tool.from_function(
    func=code_suggestions_chain.run,
    name="Write python",
    description="Useful to write good code. Pass Guidelines and a description of the code you want to write.",
    args_schema=CodeSuggestionsInput
)


In [18]:

from langchain.agents import initialize_agent

model = ChatOpenAI(temperature=0)
tools = [code_suggestion_tool, code_writing_tool, PythonREPLTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [19]:
agent.run("Write a function that calculates the first N fibonacci numbers following good guidelines")



> Entering new AgentExecutor chain...
I should start by understanding what the Fibonacci sequence is and how to calculate it. Then I can think about how to write a function that follows good coding guidelines.
Action: Code Guidelines
Action Input: Python
Observation: 1. Use meaningful variable and function names: Choose names that accurately describe the purpose or content of the variable or function. This improves code readability and makes it easier for others (or your future self) to understand the code.

2. Follow PEP 8 style guide: Adhere to the Python community's style guide, PEP 8, which provides recommendations on code formatting, naming conventions, and other coding practices. Consistent code style enhances readability and maintainability.

3. Write modular and reusable code: Break down your code into smaller, self-contained functions or classes. This promotes code reusability, improves readability, and makes it easier to test and debug specific parts of the code.

4. Avoid 

'The function to calculate the first N Fibonacci numbers following good coding guidelines is:\n\n```python\ndef fibonacci(num_terms):\n    """\n    Generates the Fibonacci sequence up to the specified number of terms.\n    """\n    if num_terms < 1:\n        return []  # Return an empty list for invalid input\n\n    fib_numbers = [0, 1]  # Initialize with the first two Fibonacci numbers\n\n    if num_terms <= 2:\n        return fib_numbers[:num_terms]  # Return the required number of terms\n\n    # Generate the remaining Fibonacci numbers using list comprehension\n    fib_numbers.extend([fib_numbers[i-1] + fib_numbers[i-2] for i in range(2, num_terms)])\n\n    return fib_numbers\n\n# Example usage:\nprint(fibonacci(10))  # Output: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]\n```'